# Analysis Notebook

In [123]:
import pandas as pd  
import matplotlib.pyplot as plt
 
pd.set_option('display.max_columns', None) # display all columns
pd.set_option("max_colwidth", 40) # sets a maximum column width
pd.set_option("display.precision", 7) # sets the number of decimals shown
pd.set_option("styler.format.precision", 3) # rounds floats

df = pd.read_csv('../data/country-data.csv')

In [122]:
def style_df(df):
    """Style dataframe"""
    cell_hover = {
        'selector': 'td:hover',
        'props': [('background-color', '#ffffb3')]
    }
    row_hover = {
        'selector': 'tr:hover',
        'props': [('background-color', '#ffffb3')]
    }
    
    numeric_columns = df.select_dtypes(include=['int8','int16','int32','int64', 'float16', 'float32', 'float64']).columns
    styler = df.style
    
    styler\
        .set_caption("INSERT CAPTION")\
        .background_gradient(axis="rows", cmap="Blues", subset=['child_mort', 'exports', 'health', 'imports', 'income', 'life_expec', 'total_fer', 'gdpp'])\
        .set_table_styles([cell_hover, row_hover])\
        .format({
            "exports": "{:}%",
            "health": "{:}%",
            "imports": "{:}%",
            })\
        .highlight_max(color="red", subset=numeric_columns)\
        .highlight_min(color="lightgreen", subset=numeric_columns)\
        .bar(subset=["inflation"], align="zero", color=["red", "lightgreen"]) # bar chart
        #.highlight_quantile(q_left=0.25, q_right=0.75, subset="imports", color="red") # highlight quantiles
        #.highlight_between(left=70, right=80, subset="life_expec", color="yellow") # highlight range of values

    return styler

style_df(df)

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.200,10.0%,7.58%,44.9%,1610,9.440,56.200,5.820,553
1,Albania,16.600,28.0%,6.55%,48.6%,9930,4.490,76.300,1.650,4090
2,Algeria,27.300,38.4%,4.17%,31.4%,12900,16.100,76.500,2.890,4460
3,Angola,119.000,62.3%,2.85%,42.9%,5900,22.400,60.100,6.160,3530
4,Antigua and Barbuda,10.300,45.5%,6.03%,58.9%,19100,1.440,76.800,2.130,12200
5,Argentina,14.500,18.9%,8.1%,16.0%,18700,20.900,75.800,2.370,10300
6,Armenia,18.100,20.8%,4.4%,45.3%,6700,7.770,73.300,1.690,3220
7,Australia,4.800,19.8%,8.73%,20.9%,41400,1.160,82.000,1.930,51900
8,Austria,4.300,51.3%,11.0%,47.8%,43200,0.873,80.500,1.440,46900
9,Azerbaijan,39.200,54.3%,5.88%,20.7%,16000,13.800,69.100,1.920,5840
